In [21]:
%load_ext autoreload
%autoreload 2

import os
import urllib

from azureml.core import  (Workspace,Run,VERSION,
                           Experiment,Datastore)

from azureml.core.runconfig import (RunConfiguration,
                                    DEFAULT_GPU_IMAGE)
from azureml.core.conda_dependencies import CondaDependencies

from azureml.core.compute import (AmlCompute, ComputeTarget)
from azureml.exceptions import ComputeTargetException

from azureml.data.data_reference import DataReference

from azureml.pipeline.core import (Pipeline, 
                                   PipelineData)
from azureml.pipeline.steps import HyperDriveStep
from azureml.pipeline.steps import PythonScriptStep

from azureml.widgets import RunDetails


print('SDK verison', VERSION)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
SDK verison 1.0.17


In [26]:
SUBSCRIPTION_ID = 'fe375bc2-9f1a-4909-ad0d-9319806d5e97'
RESOURCE_GROUP = 'amlenv_rg'
WORKSPACE_NAME = 'vienna'

PROJECT_DIR = os.getcwd()
EXPERIMENT_NAME = "customer_churn"
CLUSTER_NAME = "gpu-cluster"
DATA_DIR = os.path.join(PROJECT_DIR,'data')
SCRIPT_DIR = os.path.join(PROJECT_DIR,'train')

SOURCE_URL ='https://amlgitsamples.blob.core.windows.net/churn'
FILE_NAME = 'CATelcoCustomerChurnTrainingSample.csv'

In [27]:
ws = Workspace(workspace_name = WORKSPACE_NAME,
               subscription_id = SUBSCRIPTION_ID ,
               resource_group = RESOURCE_GROUP)

ws.write_config()

print('Workspace loaded:', ws.name)

Wrote the config file config.json to: /extdrive1/home/sasuke/dev/amlsamples/Customer_churn/aml_config/config.json
Workspace loaded: vienna


In [28]:
os.makedirs(DATA_DIR, exist_ok=True)

urllib.request.urlretrieve(os.path.join(SOURCE_URL,FILE_NAME), 
                           filename = os.path.join(DATA_DIR,FILE_NAME))

('/extdrive1/home/sasuke/dev/amlsamples/Customer_churn/data/CATelcoCustomerChurnTrainingSample.csv',
 <http.client.HTTPMessage at 0x7f7be9b7eda0>)

In [30]:
ds = ws.get_default_datastore()
data_ref = ds.upload(src_dir=DATA_DIR, target_path='churn', overwrite=True, show_progress=True)

Uploading /extdrive1/home/sasuke/dev/amlsamples/Customer_churn/data/CATelcoCustomerChurnTrainingSample.csv
Uploaded /extdrive1/home/sasuke/dev/amlsamples/Customer_churn/data/CATelcoCustomerChurnTrainingSample.csv, 1 files out of an estimated total of 1


In [19]:
cluster_name = "gpu-cluster"

try:
    cluster = ComputeTarget(ws, cluster_name)
    print(cluster_name, "found")
    
except ComputeTargetException:
    print(cluster_name, "not found, provisioning....")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6',max_nodes=1)

    
    cluster = ComputeTarget.create(ws, cluster_name, provisioning_config)
    cluster.wait_for_completion(show_output=True)

gpu-cluster found


In [15]:
cd = CondaDependencies()
cd.add_channel(channel = 'gpytorch')
cd.add_conda_package('pandas')
cd.add_conda_package('matplotlib')
cd.add_conda_package('numpy')
cd.add_conda_package('scikit-learn')

cd.add_pip_package('gpytorch')

run_config = RunConfiguration(framework="python",
                              conda_dependencies= cd)
run_config.target = cluster
run_config.environment.docker.enabled = True
run_config.environment.docker.base_image = DEFAULT_GPU_IMAGE
run_config.environment.python.user_managed_dependencies = False